# Example Selector

## 1. Set API Keys

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. Example Selector

few-shot 프롬프트를 작성하려고 할 때, 
- 예제가 많은 경우, 모든 예제를 포함할 수 없을 때 프롬프트에 포함할 예제를 선택해야 하는 경우가 발생
    - 입력에 너무 많은 토큰을 사용하게 되면 과금 문제, Context Length 문제로 효율이 좋지 않음
    - 따라서 효율적으로 예제를 운용해야함
- 또한, 답변 예시와 형식이 다양할 경우, 필요한 형식과 양식만을 가져와서 질문해야하는 경우가 생김
- `Example Selector`는 이를 담당하는 클래스
    - 사용자에 대한 질문을 입력받고, 질문과 예제의 유사도를 측정하여 유사도가 높은 예제를 선택하여 프롬프트를 구성하도록 동작

In [4]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 스티브 잡스는 몇 살에 사망했나요?
            중간 답변: 스티브 잡스는 56세에 사망했습니다.
            추가 질문: 아인슈타인은 몇 살에 사망했나요?
            중간 답변: 아인슈타인은 76세에 사망했습니다.
            최종 답변은: 아인슈타인
        """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
        """,
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 율곡 이이의 어머니는 누구인가요?
            중간 답변: 율곡 이이의 어머니는 신사임당입니다.
            추가 질문: 신사임당은 언제 태어났나요?
            중간 답변: 신사임당은 1504년에 태어났습니다.
            추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
            중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
            최종 답변은: 연산군
        """,
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 올드보이의 감독은 누구인가요?
            중간 답변: 올드보이의 감독은 박찬욱입니다.
            추가 질문: 박찬욱은 어느 나라 출신인가요?
            중간 답변: 박찬욱은 대한민국 출신입니다.
            추가 질문: 기생충의 감독은 누구인가요?
            중간 답변: 기생충의 감독은 봉준호입니다.
            추가 질문: 봉준호는 어느 나라 출신인가요?
            중간 답변: 봉준호는 대한민국 출신입니다.
            최종 답변은: 예
        """,
    },
]

In [5]:
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma(
    collection_name="example_selector",
    embedding_function=OpenAIEmbeddings()
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,                  # 예시의 목록
    OpenAIEmbeddings(),        # 의미 유사도 측정을 위한 임베딩 벡터를 생성하는 임베딩 모델 객체체
    Chroma,                    # 임베딩 벡터를 저장하고 유사성 검색을 위해 사용되는 VectorStore 클래스스
    k=1,                       # 선택할 예시의 수 (One-shot)
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

# 예시 선택택
selected_examples = example_selector.select_examples({"question": question})

print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

입력에 가장 유사한 예시:
Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?

question:
네이버의 창립자는 언제 태어났나요?
answer:
이 질문에 추가 질문이 필요한가요: 예.
            추가 질문: 네이버의 창립자는 누구인가요?
            중간 답변: 네이버는 이해진에 의해 창립되었습니다.
            추가 질문: 이해진은 언제 태어났나요?
            중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
            최종 답변은: 1967년 6월 22일
        


In [7]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}\n"
)

prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"]
)

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

chain = prompt | llm

In [8]:
print(chain.invoke(question).content)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 언제 창립되었나요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 언제 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년에 Bill Gates의 나이는 몇 살인가요?  
중간 답변: 1998년에는 Bill Gates가 43세였습니다.  
최종 답변은: 43세입니다.


이런 방식으로 프롬프팅 했을 때, 입력 토큰은 훨씬 덜 사용하면서도 비슷한 퀄리티의 답변을 얻을 수 있는 것을 확인함